# LangChain: Memory
## Outline



* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationWindowMemory
* ConversationSummaryMemoryemory

# ConversationBufferMemory

This memory allows for storing of messages and then extracts the messages in a variable

In [5]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [7]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True,
)

In [8]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [10]:
conversation.predict(input="what is 1+1")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: what is 1+1
AI:

> Finished chain.


'1+1 is equal to 2.'

In [11]:
conversation.predict(input = "what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: what is 1+1
AI: 1+1 is equal to 2.
Human: what is my name
AI:

> Finished chain.


'Your name is Andrew.'

In [12]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: what is 1+1
AI: 1+1 is equal to 2.
Human: what is my name
AI: Your name is Andrew.


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: what is 1+1\nAI: 1+1 is equal to 2.\nHuman: what is my name\nAI: Your name is Andrew."}

In [18]:
memory.save_context({"input": "Not much, just hanging out"}, {"output": "cool"})

In [19]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: what is 1+1\nAI: 1+1 is equal to 2.\nHuman: what is my name\nAI: Your name is Andrew.\nHuman: Not much, just hanging out\nAI: cool\nHuman: Not much, just hanging out\nAI: cool"}

In [20]:
from langchain.memory import ConversationBufferWindowMemory

In [21]:
memory = ConversationBufferWindowMemory(k=1)

In [23]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [24]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

# ConversationBufferWindowMemory

This memory keeps a list of the interactions of the conversations over time. It only uses the last *k* interactions

In [35]:
from langchain.memory import ConversationBufferWindowMemory

In [36]:
memory = ConversationBufferWindowMemory()

In [37]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [38]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [39]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False,
)

In [40]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [41]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [42]:
conversation.predict(input="What is my name?")

'Your name is Andrew.'

# ConversationTokenBufferMemory

This memory keeps a buffer of recent interactions in memory, and uses a token lenght, rather than a number of interactions to determine how much context to retain

In [ ]:
#!pip install tiktoken

In [43]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [46]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})


In [47]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

# ConversationSummaryMemory

Creates a summary of the previous conversation (set to a variable length in the code)

In [48]:
from langchain.memory import ConversationSummaryBufferMemory

In [49]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

In [54]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})


In [55]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [56]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [57]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'

In [59]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, the human can demonstrate the capabilities of AI in natural language processing and how it can be applied to various industries and use cases.'}

# Additional Memory types

## Vector Data Memory
Stores text (from conversation or elsewhere) in a vectory database and retreives most relevant blocks of text

## Entity Memory
Using an LLM, it remembers details about specific entities (ex. if you were conversing about a friend, you could have it remember things about that friend)

You **can** use multiple memories at one time Ex. conversational memory + entity memory

You **can** store conversations in a conventional database (like SQL)